# Basics > Cross-tabs

<div class="alert alert-info">Evaluate association between two categorical variables</div>

Cross-tab analysis is used to evaluate if categorical variables are associated. This tool is also known as chi-square or contingency table analysis

In [1]:
import matplotlib as mpl
import pyrsm as rsm

# increase plot resolution
mpl.rcParams["figure.dpi"] = 150

In [2]:
## setup pyrsm for autoreload
%reload_ext autoreload
%autoreload 2
%aimport pyrsm



### Example

The data are from a sample of 580 newspaper readers that indicated (1) which newspaper they read most frequently (USA today or Wall Street Journal) and (2) their level of income (Low income vs. High income). The data has three variables: A respondent identifier (id), respondent income (High or Low), and the primary newspaper the respondent reads (USA today or Wall Street Journal).

We will examine if there is a relationship between income level and choice of newspaper. In particular, we test the following null and alternative hypotheses:

* H0: There is no relationship between income level and newspaper choice
* Ha: There is a relationship between income level and newspaper choice

If the null-hypothesis is rejected we can investigate which cell(s) contribute to the hypothesized association. To run this test we will choose Income as the first categorical variable and Newspaper as the second. This test amounts to evaluating the size of the difference between the observed and expected frequencies. The expected frequencies are calculated using H0 (i.e., no association) as (Row total x Column Total) /  Overall Total.

In [3]:
# load example data from the pyrsm library
newspaper, newspaper_description = rsm.load_data(pkg="basics", name="newspaper")
newspaper

,id,Income,Newspaper
0,1,Low Income,USA Today
1,2,Low Income,USA Today
2,3,Low Income,USA Today
3,4,High Income,USA Today
4,5,Low Income,USA Today
...,...,...,...
575,576,Low Income,USA Today
576,577,High Income,WS Journal
577,578,High Income,WS Journal
578,579,Low Income,USA Today


In [4]:
# format the data description
rsm.md(newspaper_description)

## Newspaper readership

### Description

Data from a sample for 580 newspaper readers indicating which newspaper they read most frequently (USA today or Wall Street Journal) and their level of income (Low income vs. High income).

### Variables

A data frame with 580 observations on 3 variables

- id = Respondent identifier
- Income = A factor indicating if a respondent has High or Low income. Factor was created using a median split on income data
- Newspaper = A factor indicating the primary newspaper the respondent reads (USA today or Wall Street Journal)

To create a cross_tabs object use `rsm.cross_tabs`. The created object has attributes and methods that you can use to get the information you need. For example. the `.expected` attribute shows the expected values under the null-hypothesis as a Pandas dataframe.

In [5]:
ct = rsm.cross_tabs({"newspaper": newspaper}, "Income", "Newspaper")

In [6]:
ct.observed

Newspaper,WS Journal,USA Today,Total
Income,,,
Low Income,83,276,359
High Income,180,41,221
Total,263,317,580


In [7]:
ct.expected.round(2)

Newspaper,WS Journal,USA Today,Total
Income,,,
Low Income,162.79,196.21,359.0
High Income,100.21,120.79,221.0
Total,263.00,317.00,580.0


`cross_tabs` objects have two methods: `summary()` and `plot()`. 

In [8]:
ct.summary(output=["observed", "expected", "chisq"])


Cross-tabs
Variables: Income, Newspaper
Data     : newspaper
Null hyp : There is no association between Income and Newspaper
Alt. hyp : There is an association between Income and Newspaper

Observed:

Newspaper   WS Journal USA Today Total
Income                                
Low Income          83       276   359
High Income        180        41   221
Total              263       317   580

Expected: (row total x column total) / total

Newspaper   WS Journal USA Today  Total
Income                                 
Low Income      162.79    196.21  359.0
High Income     100.21    120.79  221.0
Total            263.0     317.0  580.0

Contribution to chi-squared: (o - e)^2 / e

Newspaper   WS Journal USA Today   Total
Income                                  
Low Income       39.11     32.45   71.55
High Income      63.53      52.7  116.23
Total           102.63     85.15  187.78

Chi-squared: 187.78 data(1), p.value < .001
0.0% of cells have expected values below 5



The (Pearson) chi-squared test evaluates if we can reject the null-hypothesis that the two variables are independent. It does so by comparing the observed frequencies (i.e., what we actually see in the data) to the expected frequencies (i.e., what we would expect to see if the two variables were independent). If there are big differences between the table of expected and observed frequencies, the chi-square value will be _large_. The chi-square value for each cell is calculated as $(o - e)^2 / e$, where $o$ is the observed frequency in a cell and $e$ is the expected frequency in that cell if the null hypothesis holds. These chi-square values can be shown by adding `output="chisq"` as an argrument to the `.summary()` method of a `cross_tab` object. The overall chi-square value is obtained by summing the chi-square values across all cells, i.e., it is the sum of the values shown in the _Contribution to chi-squared_ table.

In order to determine if the overall chi-square statistics can be considered _large_ we first determine the degrees of freedom (df). In particular: $df = (\# rows - 1) \times (\# columns - 1)$. In a 2x2 table, we have $(2-1) \times (2-1) = 1$ df. The output from `ct.summary()` shows the value of the chi-square statistic, the associated df, and the p.value associated with the test.

Remember to check the expected values: All expected frequencies are larger than 5 therefore the p.value for the chi-square statistic is unlikely to be biased. As usual we reject the null-hypothesis when the p.value is smaller 0.05. Since our p.value is very small (< .001) we can reject the null-hypothesis (i.e., the data suggest there is an association between newspaper readership and income).

We can use the provided p.value associated with the Chi-squared value of 187.783 to evaluate the null hypothesis. However, we can also calculate the critical Chi-squared value using the probability calculator in the Radiant R package. As we can see from the output below, that value is 3.841 if we choose a 95% confidence level. Because the calculated Chi-square value is larger than the critical value (187.783 > 3.841) we reject null hypothesis that `Income` and `Newspaper` are independent.

In [9]:
%reload_ext rpy2.ipython

VSCode R Session Watcher requires jsonlite, rlang. Please install manually in order to use VSCode-R.


In [10]:
%%R
Sys.getenv("R_HOME")

[1] "/opt/conda/lib/R"


In [11]:
%%R 
result <- radiant.basics::prob_chisq(df = 1, pub = 0.95)
summary(result, type = "probs")

R[write to console]: Error in library.dynam(lib, package, package.lib) : 
  shared object 'rlang.dylib' not found




Error in library.dynam(lib, package, package.lib) : 
  shared object 'rlang.dylib' not found


RInterpreterError: Failed to parse and evaluate line 'result <- radiant.basics::prob_chisq(df = 1, pub = 0.95)\nsummary(result, type = "probs")\n'.
R error message: "Error in library.dynam(lib, package, package.lib) : \n  shared object 'rlang.dylib' not found"

In [ ]:
%%R
plot(result, type = "probs")

We can also use the probability calculator to determine the p.value associated with the calculated Chi-square value. Consistent with the output from the `.summary()` method shown above this `p.value` is `< .001`.

In [ ]:
%%R
result <- radiant.basics::prob_chisq(df = 1, ub = 187.83)
summary(result)

In [ ]:
%%R
plot(result)

In addition to the numerical output provided by the `.summary()` method we can evaluate the hypothesis visually using the `.plot()` method of the `cross_tab` object. In addition to the distribution of observed and expected frequencies, we will also plot the standardized deviations (i.e., standardized differences between the observed and expected table). This measure is calculated as $(o-e)/sqrt(e)$, i.e., a score of how different the observed and expected frequencies in one cell in our table are. When a cell's standardized deviation is greater than 1.96 (in absolute value) the cell has a significant deviation from the model of independence (or no association).

In [ ]:
ct.plot(output=["observed", "expected", "dev_std"])

In the last plot above we see that all cells contribute to the association between income and readership as all standardized deviations are larger than 1.96 in absolute value (i.e., the bars extend beyond the outer dotted line in the plot).

In other words, there seem to be fewer low income respondents that read WSJ and more high income respondents that read WSJ than would be expected if the null hypothesis of no-association were true. Furthermore, there are more low income respondents that read USA today and fewer high income respondents that read USA Today than would be expected if the null hypothesis of no-association were true.

### Technical note

When one or more expected values are small (e.g., 5 or less) the p.value for the Chi-squared test is biased and it may be necessary to _collapse_ rows and/or columns and recalculate the test statistics.

# Radiant for Python App: Cross-tabs

Now lets re-run the analysis using a web-based user interface. Select `Income` as the first categotical variable and `Newspaper` as the second. This test amounts to evaluating the size of the difference between the observed and expected frequencies. The expected frequencies are calculated using H0 (i.e., no association) as (Row total x Column Total) /  Overall Total. 

Select `Observed`, `Expected`, and `Chi-squared` as the output tables to create. The (Pearson) chi-squared test evaluates if we can reject the null-hypothesis that the two variables are independent. It does so by comparing the observed frequencies (i.e., what we actually see in the data) to the expected frequencies (i.e., what we would expect to see if the two variables were independent). If there are big differences between the table of expected and observed frequencies, the chi-square value will be _large_. The chi-square value for each cell is calculated as $(o - e)^2 / e$, where $o$ is the observed frequency in a cell and $e$ is the expected frequency in that cell if the null hypothesis holds. These chi-square values can be shown by adding `output="chisq"` as an argrument to the `summary()` method of a `cross_tab` object. The overall chi-square value is obtained by summing the chi-square values across all cells, i.e., it is the sum of the values shown in the _Contribution to chi-squared_ table.

> Note: The app will continue running until you press the `Stop` button in the app navigation bar or the stop button next to the notebook cell

In [12]:
rsm.radiant.cross_tabs(
    {"newspaper": newspaper}, {"newspaper": newspaper_description}, open=True
)


Listening on http://0.0.0.0:8000
Pyrsm and Radiant are open source tools and free to use. If you
are a student or instructor using pyrsm or Radiant for a class,
as a favor to the developers, please send an email to
<radiant@rady.ucsd.edu> with the name of the school and class.


```python
import pyrsm as rsm

# newspaper = pd.read_parquet('newspaper.parquet')
ct = rsm.cross_tabs(data={"newspaper": newspaper}, var1="Income", var2="Newspaper")
ct.summary(output=["observed", "expected", "chisq"])

```

## Video Tutorials

To review the same analysis in Radiant for R, copy-and-paste the full command below into the RStudio console (i.e., the bottom-left window) and press return to gain access to all materials used in the hypothesis testing module of the <a href="https://www.youtube.com/playlist?list=PLNhtaetb48EdKRIY7MewCyvb_1x7dV3xw">Radiant Tutorial Series</a>:

usethis::use_course("https://www.dropbox.com/sh/0xvhyolgcvox685/AADSppNSIocrJS-BqZXhD1Kna?dl=1")

[Cross-tabs Hypothesis Test](https://youtu.be/MEt8vO5FNYQ){target="_blank"}

* This video demonstrates how to investigate associations between two categorical variables by a cross-tabs hypothesis test
* Topics List:
    - Setup a hypothesis test for cross-tabs in Radiant
    - Explain how observed, expected and contribution to chi-squared tables are constructed
    - Use the p.value and critical value to evaluate the hypothesis test